# Excel files
By the end of this lecture you will be able to:
- read an Excel worksheet into a `DataFrame`
- choose which engine to use when reading `DataFrames`
- read multiple Excel worksheets into a `dict`
- writing to an Excel worksheet
- adding conditional formatting and sparklines

In [ ]:
from pathlib import Path

import polars as pl

In [ ]:
csv_file = "../data/titanic.csv"

## Creating an Excel file
In this IO section of the course we use the CSV datasets to create datasets in other formats. We write these datasets to a new `data_files` sub-directory of this directory (i.e. 07_io).

We first create a simple Excel file with one worksheet from the Titanic CSV file. 

To write to an Excel file we need to have the XlsxWriter package installed. XlsxWriter should have been installed when you initially created the virtual environment but you can `pip` install it now if you don't have it.

We set the path to our CSV file and the Excel file that we will create in a sub-directory

To create the path to the Excel file we:
- create a `Path` object using the `Path` function from Python's built-in `pathlib` module
- call `mkdir` on the `Path` object to create the sub-directory and any parent sub-directories required
- create a `Path` object to the new Excel file

In [ ]:
# Specify a directory to hold the excel files as a Path object
excel_titanic_dir = Path('data_files/excel/titanic')
# Set the file name of the Excel file
excelFile = "titanic.xlsx"
# Create the Titanic sub-directory if it doesn't exist already
excel_titanic_dir.mkdir(parents=True,exist_ok=True)
# Set the path to the Titanic excel file
titanic_excel_path = excel_titanic_dir / excelFile

We now create the Excel file at this `Path`.

We read the CSV into a `DataFrame` and write it to a .xlsx file

In [ ]:
df = pl.read_csv(csv_file)
df.write_excel(titanic_excel_path)

We cover writing to Excel in more detail below.

## Reading from a spreadsheet
In the simplest cases we can just read the first sheet in the file with `pl.read_excel`

In [ ]:
df = pl.read_excel(titanic_excel_path)
df.head(2)

## Specifying the output `DataFrame`

Reading Excel files happens in eager mode only, we cannot do a lazy scan of an Excel file.

However, we can reduce memory usage by specifying which columns to read

In [ ]:
(
    pl.read_excel(titanic_excel_path,columns=["Name","Age"])
    .head(2)
)

We can also specify column names as a sequence of integers

In [ ]:
(
    pl.read_excel(titanic_excel_path,columns=[3,5])
    .head(2)
)

As for CSVs we can also set the number of rows used to infer the schema with `infer_schema_length` and override the inferred schemas with `schema_overrides`

In [ ]:
(
    pl.read_excel(titanic_excel_path,
                  infer_schema_length=200,
                  schema_overrides={"Pclass":pl.Int32}
    )
    .head(2)
)

## Choosing a worksheet
We can choose a worksheet with either integer id numbers or names.

### Specifying with id numbers
We specify the sheet by numbers with the `sheet_id` argument.
- By default `sheet_id = 1` and Polars reads the first worksheet
- If we set `sheet_id = 0` Polars returns all sheets as a `dict` that maps string sheet names to `DataFrames`

In [ ]:
sheet_dict = (
    pl.read_excel(titanic_excel_path,sheet_id=0)
)
sheet_dict.keys()

### Specifying with sheet name
By default there is no `sheet_name` and the `sheet_id = 1` argument controls the behaviour. We can instead specify the sheet by name with the `sheet_name` argument. 

In [ ]:
(
    pl.read_excel(
        titanic_excel_path,
        sheet_name="Sheet1"
    )
    .head(2)
)

### Choosing the engine
Polars uses third party libraries to parse the Excel file. The library used to parse the Excel file is called the *engine* in Polars. The current options are:
- calamine (the default)
- xlsx2csv (the pre version 1.0 default)
- openpyxl
- pyxlsb (for Excel binary worksbooks only)

The calamine engine is much faster than the other options and should be used if possible. However, it is based on newer packages and has fewer features for parsing data.

To specify the engine you pass the `engine` argument. Here we specify the xlsx2csv engine

In [ ]:
df = pl.read_excel(titanic_excel_path,engine="xlsx2csv")
df.head(2)

We look at the two primary engines in more detail 

### calamine
The calamine engine relies on two Rust crates (where crates are the Rust equivalent of Python packages):
- calamine which parses the spreadsheet XML and
- fastexcel which converts the data to an Apache Arrow table

To use calamine you must also have the fastexcel library installed. If this was not installed when you started the course you can do so now with `pip install fastexcel`. 

There are (limited) docs about the fastexcel library here: https://pypi.org/project/fastexcel/

If we want to pass parsing options (e.g. to set the header row or skip rows) to the calamine reader we pass these as a `dict` to `read_options`. We can pass arguments from the [`load_sheet` method in fastexcel](https://fastexcel.toucantoco.dev/fastexcel.html#ExcelReader.load_sheet). Here for example we pass `skip_rows` to skip the first row after the header

In [ ]:
(
    pl.read_excel(
        titanic_excel_path,
        read_options={"skip_rows":1}
    )
    .head(2)
) 

Many of the arguments to `load_sheet` are covered by the standard arguments to `pl.read_excel`, for example if we use the `columns` argument in `pl.read_excel` Polars just passes this to `use_columns` when using calamine internally. It's better to stick to the `pl.read_excel` arguments as these still work if we change the engine.

#### xlsx2csv
This was the default parses before version 1.0 of Polars. When we call `pl.read_excel` with the `xlsx2csv` engine:
- Polars passes the path to the Excel file to the xlsx2csv package 
- `xlsx2csv` parses the XML and converts it to a CSV in bytes in-memory
- Polars parses the in-memory CSV with `pl.read_csv`

##### Parsing the XML with xlsx2csv
We can pass arguments to xlsx2csv to control how it parses the XML. These options include:
- specifying the date format with `DATEFORMAT %Y/%m/%d`
- specifying the format for floats with `FLOATFORMAT %.15f`
- skip empty lines

See https://github.com/dilshod/xlsx2csv for the full set of options.

We pass these arguments as a `dict` to the `engine_options` argument (see below for how we use `read_options` with `xlsx2csv`)

In [ ]:
(
    pl.read_excel(
        titanic_excel_path,
        engine="xlsx2csv",
        engine_options ={"skip_empty_lines": True}
    )
    .head(2)
)

Once `xlsx2csv` has created the CSV we can pass the same arguments that we would pass to `pl.read_csv` in `read_options`

In this example we rename the first column using `new_columns`

In [ ]:
(
    pl.read_excel(
        titanic_excel_path,
        engine="xlsx2csv",
        read_options =
            {
                "new_columns":["Id"]
            }
    )
    .head(2)
)

For clarity, this step of using `pl.read_csv` internally only applies to the `xlsx2csv` reader, it does not apply to `calamine`.

### Which engine should I use?
Use calamine if possible as it is much faster.

However, if your data does not load correctly using calamine (e.g. the dtypes are not correctly inferred) then there is more documentation explaining how to manage this with the xlsxcsv engine. Although in many cases it may be easier and faster to just read the data into Polars as strings and then fix it in Polars, for example to use `str.strptime` to parse tricky dates as we see in the next section.

Overall, try the calamine engine but if you have problems then use xlsx2csv


Parsing the XML in Excel files is always slow. Consider working with Parquet if possible, and even CSVs are much faster than parsing spreadsheets!

## Writing to a spreadsheet

As we saw above to write a `DataFrame` to Sheet1 of a new .xlsx file we call `write_excel`

In [ ]:
(
    df
    .write_excel(
        titanic_excel_path
    )
)

### Formatting the worksheet
We have a lot of control over how the worksheet looks

For example we can:
- use built-in Excel table styles with the `table_style` argument
- pass a `dict` mapping column names to column widths in pixels with `column_widths`


In [ ]:
(
    df
    .write_excel(
        titanic_excel_path,
        table_style='Table Style Medium 2',
        column_widths = {col:100 for col in df.columns},
        column_formats = {'Age':"0.000"}
    )
)


Instead of passing `column_widths` manually we can also use the `autofit` argument

In [ ]:
(
    df
    .write_excel(
        titanic_excel_path,
        autofit=True
    )
)


### Formatting values
We can set individual column formats in a `dict` with the `column_formats`. We use the patterns that are found when you format a column in Excel under the `Custom` option

In [ ]:
(
    df
    .write_excel(
        titanic_excel_path,
        table_style='Table Style Medium 2',
        column_formats = {'Age':"0.000"}
    )
)


To format floats it may be easier to use the `float_precision` argument

In [ ]:
(
    df
    .write_excel(
        titanic_excel_path,
        table_style='Table Style Medium 2',
        float_precision=4
    )
)


### Conditional formatting
We can apply conditional formatting using the options allowed by Xlswriter. For example, we can have a bar chart in the `Age` column and a 3-color scale for the `Fare` column

In [ ]:
(
    df
    .write_excel(
        titanic_excel_path,
        table_style='Table Style Medium 2',
        autofit=True,
        float_precision=3,
        conditional_formats = {'Age':'data_bar','Fare':'3_color_scale'}
    )
)


See the Xlswriter docs for more info:https://xlsxwriter.readthedocs.io/working_with_conditional_formats.html

### Sparklines
We can add sparklines to give a simple visualisation of trends along a row.

In this example we first create a `DataFrame` with statistics for each class along a row. We make sparklines to show how these vary across classes.

To get the data in the right format we need to do some reshaping with `melt` and `pivot` first. In the example below I have commented out the `write_excel` part so you can see these transformations. Uncomment the `write_excel` part when you want to write the output to the file

In [ ]:
(
    df
    .group_by('Pclass')
    .agg(
        pl.col('Age').mean(),
        pl.col('Fare').mean(),
        pl.col('Survived').count()
    )
    .with_columns(index=pl.lit(0))
    .unpivot(index=['Pclass'])
    .pivot(index='variable',on='Pclass',values='value',aggregate_function='first')
    .pipe(lambda df: df.select('variable','1','2','3'))
    # .write_excel(
    #     excel_titanic_dir / 'titanic_groupby.xlsx',
    #     table_style='Table Style Medium 2',
    #     autofit=True,
    #     sparklines={"trend": ["1","2","3"]}
    # )
)


Read more about sparklines in the API docs: https://pola-rs.github.io/polars/py-polars/html/reference/api/polars.DataFrame.write_excel.html#polars.DataFrame.write_excel

## Exercises
In the exercises you will develop your understanding of:
- writing to an excel file
- adding formatting and sparklines
- passing arguments when reading an excel file

### Exercise 1
Create a `DataFrame` from the NYC taxi extract and write it to an Excel file called `nyc.xlsx`

In [ ]:
nyccsv_file = "../data/nyc_trip_data_1k.csv"
# Make a Path variable to write the 
nycExcelFile = Path('data_files/excel/nyc')
nycExcelFile.mkdir(parents=True,exist_ok=True)
(
    pl.read_csv(nyccsv_file)
    <blank>
)

Write the `DataFrame` to the same file but with a bar chart in the `trip_distance` column and a colormap in the `tip_amount` column

In [ ]:
(
    pl.read_csv(nyccsv_file)
    .write_excel(
        <blank>
    )
)

Create a `DataFrame` from the `nyc.xlsx` file with the `pickup` and `dropoff` columns as datetime dtypes using the `xlsx2csv` engine

In [ ]:
(
    pl.read_excel(
        nycExcelFile / 'nyc.xlsx',
        <blank>
    )
    .head()
)

### Exercise 2
Get the average of all the floating point columns by day of the week of `pickup` (hint use `.dt.weekday` in the `groupby` expression). Sort the output by the day of the week

In [ ]:
(
    pl.read_csv(nyccsv_file,try_parse_dates=True)
    <blank>
)

Reshape the output so that there is one column of variable names and a column for each day of the week

Write the reshaped output to an excel file called `nyc_day_of_week.xlsx` in the same directory as above. Add sparklines to show the trend across the days of the week. Ensure the `variable` column is wide enough to be fully legible

## Solutions

### Solution to Exercise 1
Create a `DataFrame` from the NYC taxi extract and write it to an Excel file called `nyc.xlsx`

In [ ]:
nyccsv_file = "../data/nyc_trip_data_1k.csv"
# Make a Path variable to write the 
nycExcelFile = Path('data_files/excel/nyc')
nycExcelFile.mkdir(parents=True,exist_ok=True)
(
    pl.read_csv(nyccsv_file)
    .write_excel(nycExcelFile / 'nyc.xlsx')
)

Write the `DataFrame` to the same file but with a bar chart in the `trip_distance` column and a colormap in the `tip_amount` column

In [ ]:
(
    pl.read_csv(nyccsv_file)
    .write_excel(
        nycExcelFile / 'nyc.xlsx',
        conditional_formats = {'trip_distance':'data_bar','tip_amount':'3_color_scale'}
    )
)

Create a `DataFrame` from the `nyc.xlsx` file with the `pickup` and `dropoff` columns as datetime dtypes using the `xlsx2csv` engine

In [ ]:
(
    pl.read_excel(
        nycExcelFile / 'nyc.xlsx',
        engine="xlsx2csv",
        read_options={"try_parse_dates":True}
    )
    .head()
)

There is no corresponding option for calamine so we parse the strings with `str.strptime` in Polars

In [ ]:
(
    pl.read_excel(
        nycExcelFile / 'nyc.xlsx',
    )
    .with_columns(
      pl.col("pickup","dropoff").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%.6f"),
    )
    .head(2)
)

### Solution to Exercise 2
Get the average of all the floating point columns by day of the week of `pickup` (hint use `.dt.weekday` in the `groupby` expression). Sort the output by the day of the week

In [ ]:
(
    pl.read_csv(nyccsv_file,try_parse_dates=True)
    .group_by(
        pl.col('pickup').dt.weekday()
    )
    .agg(
        pl.col(pl.Float64).mean()
    )
    .sort("pickup")
)

Reshape the output so that there is one column of variable names and a column for each day of the week

In [ ]:
(
    pl.read_csv(nyccsv_file,try_parse_dates=True)
    .group_by(
        pl.col('pickup').dt.weekday()
    )
    .agg(
        pl.col(pl.Float64).mean()
    )
    .sort("pickup")
    .unpivot(index="pickup")
    .pivot(index="variable",on="pickup",values="value",aggregate_function="first"
    )
)

Write the reshaped output to an excel file called `nyc_day_of_week.xlsx` in the same directory as above. Add sparklines to show the trend across the days of the week. Ensure the `variable` column is wide enough to be fully legible

In [ ]:
(
    pl.read_csv(nyccsv_file,try_parse_dates=True)
    .group_by(
        pl.col('pickup').dt.weekday()
    )
    .agg(
        pl.col(pl.Float64).mean()
    )
    .sort("pickup")
    .unpivot(index="pickup")
    .pivot(index="variable",on="pickup",values="value",aggregate_function="first")
    .write_excel(
        workbook = nycExcelFile / "nyc_day_of_week.xlsx",
        sparklines={"trend":[str(idx) for idx in range(1,8)]},
        column_widths = {'variable':100}
    )  
)